# Block 40: Demonstration - Part 1 (Solution)
## __Dropout Regularization__

Dropout is a technique where:

- Randomly selected neurons are ignored during training. They are "dropped out" randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass, and any weight updates are not applied to the neuron on the backward pass.

- If neurons are randomly dropped out of the network during training, other neurons will have to step in and handle the representation required to make predictions for the missing neurons. This is believed to result in multiple independent internal representations being learned by the network.

- The effect is that the network becomes less sensitive to the specific weights of neurons. This, in turn, results in a network that is capable of better generalization and is less likely to overfit the training data.



## Steps to Be Followed:
1. Importing the required libraries
2. Reading a CSV file into a DataFrame
3. Creating the dummies
4. Preparing the data for modeling
5. Performing K-fold cross-validation and model training
6. Calculating the error

### Step 1: Importing the Required Libraries

- Import the libraries for data preprocessing, including z-score standardization using **scipy.stats.zscore**, and data manipulation using **pandas**. Also imports libraries for model evaluation, such as metrics from **sklearn** and train-test splitting from **sklearn.model_selection**.
- Import the necessary components from TensorFlow Keras (**Sequential** and **Dense**) to build a neural network model. These components allow for the creation of a sequential model with dense layers and activation functions.

In [1]:
import pandas as pd  # Import the pandas library for data handling
from scipy.stats import zscore  # Import the zscore function from scipy.stats
import numpy as np  # Import the numpy library for numerical operations
from sklearn import metrics  # Import the scikit-learn metrics module
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data
from tensorflow.keras.models import Sequential  # Import Sequential model from Keras
from tensorflow.keras.layers import Dense, Activation  # Import Dense and Activation layers from Keras

### Step 2: Reading a CSV File into a DataFrame
- The code reads a CSV file from a given URL and stores it in a Pandas DataFrame by using **na_values** to replace specified values with missing values.

In [2]:
# dataset link : "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv

In [3]:
# Read the data set
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

In [4]:
# Display the first 5 rows of the DataFrame using the .head() method
df.head(5)


id job area   income     aspect  subscriptions  dist_healthy  save_rate  \
0   1  vv    c  50876.0  13.100000              1      9.017895         35   
1   2  kd    c  60369.0  18.625000              2      7.766643         59   
2   3  pe    c  55126.0  34.766667              1      3.632069          6   
3   4  11    c  51690.0  15.808333              1      5.372942         16   
4   5  kl    d  28347.0  40.941667              3      3.822477         20   

   dist_unhealthy  age  pop_dense  retail_dense     crime product  
0       11.738935   49   0.885827      0.492126  0.071100       b  
1        6.805396   51   0.874016      0.342520  0.400809       c  
2       13.671772   44   0.944882      0.724409  0.207723       b  
3        4.333286   50   0.889764      0.444882  0.361216       b  
4        5.967121   38   0.744094      0.661417  0.068033       a

**Observation**
- The output appears to be a tabular representation of a dataset with various columns.
- Each row represents a sample or instance, while each column represents a different attribute or feature of that instance.
- The columns contain information such as the ID, job, area, income, aspect, subscriptions, dist_healthy, save_rate, dist_unhealthy, age, pop_dense, retail_dense, crime, and product.
- The values in the columns represent specific measurements or categories related to each attribute.

### Step 3: Creating the Dummies

- The code uses the **pd.get_dummies()** function to convert categorical columns **'job'** and **'area'** into dummy variables, which represent the presence or absence of each category as binary values.

- The resulting dummy variables are concatenated with the original DataFrame **df** using **pd.concat()**, which adds the dummy variables as new columns.

- Finally, the original categorical columns **'job'** and **'area'** are dropped from the DataFrame using the **df.drop()** function with the **axis=1** parameter set to remove columns. This ensures that only the dummy variables remain in the DataFrame.


In [5]:
# Concatenate one-hot encoded columns for the 'job' column and drop the original 'job' column
df = pd.concat([df, pd.get_dummies(df['job'], prefix="job")], axis=1)
df.drop('job', axis=1, inplace=True)

# Concatenate one-hot encoded columns for the 'area' column and drop the original 'area' column
df = pd.concat([df, pd.get_dummies(df['area'], prefix="area")], axis=1)
df.drop('area', axis=1, inplace=True)


- The missing values in the **'income'** column of the DataFrame **'df'** are filled with the median value of the **'income'** column.

In [6]:
# Calculate the median of the 'income' column
med = df['income'].median()

# Fill missing (NaN) values in the 'income' column with the calculated median value
df['income'] = df['income'].fillna(med)


- The specified columns **('income', 'aspect', 'save_rate', 'age', 'subscriptions')** in the DataFrame **df** are standardized using z-score normalization, which transforms the values to have zero mean and unit variance.

In [7]:
# Standardize (Z-score normalize) the 'income' column
df['income'] = zscore(df['income'])

# Standardize (Z-score normalize) the 'aspect' column
df['aspect'] = zscore(df['aspect'])

# Standardize (Z-score normalize) the 'save_rate' column
df['save_rate'] = zscore(df['save_rate'])

# Standardize (Z-score normalize) the 'age' column
df['age'] = zscore(df['age'])

# Standardize (Z-score normalize) the 'subscriptions' column
df['subscriptions'] = zscore(df['subscriptions'])


### Step 4: Preparing the Data for Modeling
- The code selects the relevant columns from the DataFrame df by dropping the **'product'** and **'id'** columns and assigns them to the variable **x_columns**.
- The code creates dummy variables for the **'product'** column using one-hot encoding and assigns the column names to the variable products. The target variable **'y'** is assigned the corresponding dummy variable values.

In [8]:
# Create a variable 'x_columns' to store column names, excluding 'product' and 'id'
x_columns = df.columns.drop('product').drop('id')

# Extract the feature values from the DataFrame 'df' for the selected columns
x = df[x_columns].values

# Create one-hot encoded 'dummies' for the 'product' column
dummies = pd.get_dummies(df['product'])

# Store the names of the product categories in 'products'
products = dummies.columns

# Extract the target variable 'y' as one-hot encoded values
y = dummies.values


In [14]:
print(y)
[False,  True, False, ..., False, False, False] -> 'categorical_crossentropy'
[False,  True, False, ..., False, False, False]
[False,  True, False, ..., False, False, False]
[False,  True, False, ..., False, False, False]


print(y)
1
2
3
7
8
9
-> 'sparse_categorical_crossentropy'

SyntaxError: invalid syntax (<ipython-input-14-4355d021dd4c>, line 1)

In [15]:
y.shape[1]

7

### Step 5: Performing K-Fold Cross-Validation and Model Training
- Train a model using K-fold cross-validation with 5 folds.
- The model consists of a sequential neural network with two hidden layers, using ReLU activation for the first hidden layer and L1 regularization for the second hidden layer.
- Dropout is applied to the first hidden layer to prevent overfitting.
- The model is trained using the Adam optimizer and categorical cross-entropy loss function.
- The accuracy of each fold is calculated and printed.

The **verbose** parameter is used in the model.fit() function. It controls the amount of information printed during the training process.
When **verbose** is set to 0 (as in this code), it means "silent mode," and no output will be printed during training.

In [9]:
# Import necessary libraries and modules
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
import numpy as np
from sklearn import metrics

# Create a KFold cross-validation object with 5 folds, shuffling the data, and a random seed
kf = KFold(5, shuffle=True, random_state=42)

# Initialize empty lists to store out-of-sample (oos) true labels and predictions
oos_y = []
oos_pred = []
fold = 0

# Loop through each fold in the cross-validation
for train, test in kf.split(x):
    fold += 1
    print(f"Fold #{fold}")

    # Split the data into training and testing sets based on the fold indices
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Create a Sequential neural network model
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu'))  # Hidden layer 1
    model.add(Dropout(0.5))
    model.add(Dense(25, activation='relu', activity_regularizer=regularizers.l1(1e-4)))  # Hidden layer 2
    # Usually, dropout is not added after the final hidden layer
    model.add(Dropout(0.5))

    model.add(Dense(y.shape[1], activation='softmax'))  # Output layer

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # Train the model on the training data
    model.fit(x_train, y_train, verbose=0, epochs=10)

    # Make predictions on the test data
    pred = model.predict(x_test)
    oos_y.append(y_test)

    # Convert raw probabilities to the chosen class (highest probability)
    pred = np.argmax(pred, axis=1)
    oos_pred.append(pred)

    # Measure the accuracy of this fold's predictions
    y_compare = np.argmax(y_test, axis=1)  # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")


Fold #1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

**Observation**
- The output shows the accuracy scores for each fold of the cross-validation process:

  - Fold Scores: The output displays the fold number (e.g., Fold #1) and the corresponding accuracy score (e.g., 0.67) for each fold. The accuracy score represents the proportion of correctly predicted labels to the total number of labels in the test set. Higher accuracy scores indicate better performance of the model on the test data.

  - Performance Variation: The output demonstrates that the model's performance varies across different folds. This variation can provide insights into the stability and robustness of the model. The accuracy scores range from 0.61 to 0.67, suggesting that the model performs reasonably well but with some degree of variability across different subsets of the data.

### Step 6: Calculating the Error
- The code calculates the final accuracy score and creates a DataFrame combining the original data with the true values and predicted values.

In [ ]:
# Concatenate the out-of-sample (oos) true labels and predictions for all folds
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

# Convert the oos true labels to the chosen class (highest probability) for accuracy calculation
oos_y_compare = np.argmax(oos_y, axis=1)

# Calculate the accuracy score for the combined predictions
score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")

# Create DataFrames for the out-of-sample true labels and predictions
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)

# Concatenate the original DataFrame 'df' with the out-of-sample true labels and predictions
oosDF = pd.concat([df, oos_y, oos_pred], axis=1)



**Observation**
- The final accuracy score achieved by the model is 0.6315(as given above).